## first simple domain adaptation (noise + lobe segments)

In [ ]:
import numpy as np
import nibabel as nib
from scipy.ndimage import gaussian_filter, binary_erosion
import yaml
import matplotlib.pyplot as plt
import time

In [ ]:
# histogram
def histogram(adapted_vol, orig_vol, lung_mask, out_dir, hash):
    voxels1 = adapted_vol[lung_mask].ravel()
    voxels2 = orig_vol[lung_mask].ravel()
    plt.hist(voxels2, bins=200, label='orig_vol', histtype='step')
    plt.hist(voxels1, bins=200, label='adapted_vol', histtype='step')
    plt.legend()
    plt.yscale('log')
    plt.savefig(f'{out_dir}/hist/{hash}_histogram.png')

In [ ]:
vessel_dir = '/home/ahaas/airway-seg/vessel_graph_generation/datasets/blood_vessels/20230907_184749_1f6a6f24-df7a-4bd5-8612-a8c8f5446d98'
airway_dir = '/home/ahaas/airway-seg/vessel_graph_generation/datasets/airways/20230907_201008_0f134eb7-0178-48b5-8313-7474de12e727'
out_dir = '/home/ahaas/airway-seg/vessel_graph_generation/datasets/dataset_4'
orig_dir = '/home/shared/Data/ATM22/train/'

In [ ]:
# init: load niftis
start = time.time()
hash = airway_dir.split('/')[-1]
with open(f'{airway_dir}/config.yml', "r") as filepath:
    config = yaml.safe_load(filepath)
scan = config['Forest']['segment_root_path'][-13:-10]

airway = nib.load(airway_dir + '/volume.nii.gz').get_fdata().astype(np.float32)
airway_mask = airway > 0 #biggest mask

lumen = nib.load(airway_dir + '/lumen.nii.gz').get_fdata().astype(np.float32)
lumen_mask = lumen > 0 #small mask

#smallest mask
label_mask = nib.load(airway_dir + '/airways.nii.gz').get_fdata().astype(np.uint8) >= 1

vessels = nib.load(vessel_dir + '/volume.nii.gz').get_fdata().astype(np.float32)
vessels_mask = vessels > 0

lung_mask = nib.load(airway_dir + '/lung.nii.gz').get_fdata().astype(np.uint8)
lung_mask = lung_mask >= 1

orig_img = nib.load(f'{orig_dir}/images/ATM_{scan}_0000.nii.gz')
orig_vol = orig_img.get_fdata()
affine = orig_img.affine
del orig_img

combined_mask = np.logical_or(airway_mask, vessels_mask)
parenchyma_mask = np.logical_and(lung_mask, combined_mask == False)
blend_mask = binary_erosion(combined_mask)
print(combined_mask.astype(int).sum(), blend_mask.astype(int).sum())

end = time.time()
print(end-start, 'init')
start = end

In [ ]:
# HU mapping
start = time.time()

airway_mapping = lambda x: 220*(x+0.2)**2-915
mapped_airway = airway_mapping(airway[airway_mask])
lumen_mapping = lambda x: 205*(x-1.35)**2-1050
mapped_lumen = lumen_mapping(lumen[label_mask])

vessel_mapping = lambda x: 450 * (x+0.4)**2.5 - 980
mapped_vessels = vessel_mapping(vessels[vessels_mask])
vessel_noise = np.random.normal(-10, 5, np.sum(vessels_mask))
mapped_vessels += vessel_noise

adapted_volume = np.zeros_like(airway)
adapted_volume[vessels_mask] = mapped_vessels
adapted_volume[airway_mask] = mapped_airway
adapted_volume[label_mask] = mapped_lumen

end = time.time()
print(end-start, 'HU mapping')

In [ ]:
shape = airway.shape
noise = np.random.uniform(0, 1, shape)
noise_smoothed = gaussian_filter(noise, sigma=(0.8, 0.8, 0.9))

plt.figure(figsize=(25, 10))
plt.subplot(2, 4, 1)
plt.hist(orig_vol[100:130, 230:260, 330:360].ravel(), bins=100)
plt.subplot(2, 4, 2)
plt.imshow(orig_vol[100:130, 230:260, 345])
plt.subplot(2, 4, 3)
plt.imshow(orig_vol[100:130, 245, 330:360])
plt.subplot(2, 4, 4)
plt.imshow(orig_vol[115, 230:260, 330:360])
plt.subplot(2, 4, 5)
plt.hist(noise_smoothed[100:130, 230:260, 330:360].ravel(), bins=100)
plt.subplot(2, 4, 6)
plt.imshow(noise_smoothed[100:130, 230:260, 345])
plt.subplot(2, 4, 7)
plt.imshow(noise_smoothed[100:130, 245, 330:360])
plt.subplot(2, 4, 8)
plt.imshow(noise_smoothed[115, 230:260, 330:360])
plt.show()

In [ ]:
min, max = noise_smoothed.min(), noise_smoothed.max()
noise_smoothed = (noise_smoothed-min) / (max-min) #normalize to [0,1]

bins = 1000 #max-min
hist, _ = np.histogram(noise_smoothed, bins=bins)
cdf1 = hist.cumsum().astype(np.float64)
cdf1 /= cdf1.max() #normalize to [0,1]

x = np.linspace(0, 1, bins) 
mu = -2.5 #-1.7
sig = 0.3 #0.4
target_pdf = 1/(x*sig*np.sqrt(2*np.pi)+1e-5)*np.exp(-(np.log(x+1e-5)-mu)**2/(2*sig**2))
target_cdf = target_pdf.cumsum()
target_cdf /= target_cdf.max() #normalize to [0,1]

min, max = 0, 1
mapping_function = np.interp(cdf1, target_cdf, np.arange(min, max, (max-min)/bins))

noise_equalized = np.interp(noise_smoothed.ravel(), np.arange(0, 1, 1/bins), mapping_function)
noise_equalized = noise_equalized.reshape(noise_smoothed.shape)

noise_equalized_scaled = noise_equalized * 1250 - 1000


plt.figure(figsize=(25, 20))
plt.subplot(4, 4, 1)
plt.hist(orig_vol[100:130, 230:260, 330:360].ravel(), bins=100)
plt.xlim((-1000, 0))
plt.subplot(4, 4, 2)
plt.imshow(orig_vol[100:130, 230:260, 345])
plt.subplot(4, 4, 3)
plt.imshow(orig_vol[100:130, 245, 330:360])
plt.subplot(4, 4, 4)
plt.imshow(orig_vol[115, 230:260, 330:360])
plt.subplot(4, 4, 5)
plt.hist(noise_smoothed[100:130, 230:260, 330:360].ravel(), bins=100)
plt.subplot(4, 4, 6)
plt.imshow(noise_smoothed[100:130, 230:260, 345])
plt.subplot(4, 4, 7)
plt.imshow(noise_smoothed[100:130, 245, 330:360])
plt.subplot(4, 4, 8)
plt.imshow(noise_smoothed[115, 230:260, 330:360])
plt.subplot(4, 4, 9)
plt.hist(noise_equalized[100:130, 230:260, 330:360].ravel(), bins=100)
plt.subplot(4, 4, 10)
plt.imshow(noise_equalized[100:130, 230:260, 345])
plt.subplot(4, 4, 11)
plt.imshow(noise_equalized[100:130, 245, 330:360])
plt.subplot(4, 4, 12)
plt.imshow(noise_equalized[115, 230:260, 330:360])
plt.subplot(4, 4, 13)
plt.hist(noise_equalized_scaled[100:130, 230:260, 330:360].ravel(), bins=100)
plt.xlim((-1000, 0))
plt.subplot(4, 4, 14)
plt.imshow(noise_equalized_scaled[100:130, 230:260, 345])
plt.subplot(4, 4, 15)
plt.imshow(noise_equalized_scaled[100:130, 245, 330:360])
plt.subplot(4, 4, 16)
plt.imshow(noise_equalized_scaled[115, 230:260, 330:360])
plt.show()

In [ ]:
# blend all
start = time.time()

blend_factor = 0.9

final_volume = noise_equalized_scaled
final_volume = np.where(combined_mask, adapted_volume, final_volume) # add airways and vessels
final_volume = np.where(lung_mask, final_volume, orig_vol) # add background

#final_volume = np.where(lung_mask, final_volume, orig_vol) # add background
#final_volume = gaussian_filter(final_volume, sigma=(0, 0, 0.5))
#final_volume = np.where(blend_mask, adapted_volume*blend_factor+noise_equalized_scaled*(1-blend_factor), final_volume) # blend edge of airways and vessels
#final_volume = np.where(lung_mask, final_volume, orig_vol) # add background

end = time.time()
print(end-start, 'blend all')
start = end

In [ ]:
# histogram orig and adapted
start = time.time()
histogram(final_volume, orig_vol, lung_mask, out_dir, hash)

end = time.time()
print(end-start, 'histogram')
start = end

In [ ]:
# saving
start = time.time()
nifti = nib.Nifti1Image(final_volume.astype(np.int16), affine)
nib.save(nifti, f"{out_dir}/images/{hash}_volume.nii.gz")

# vessel
vessels_seg = vessels > 0.25
vessels_seg = np.where(lumen_mask, 0, vessels_seg)
vessels_seg = np.logical_and(vessels_seg, lung_mask == False)
nifti = nib.Nifti1Image(vessels_seg.astype(np.uint8), affine)
nib.save(nifti, f"{out_dir}/labels/{hash}_vessels.nii.gz")

# airways
label_mask = np.logical_and(label_mask, lung_mask == False)
nifti = nib.Nifti1Image(label_mask.astype(np.uint8), affine)
nib.save(nifti, f"{out_dir}/labels/{hash}_airways.nii.gz")

end = time.time()
print(end-start, 'saving')